In [8]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset, random_split
import pandas as pd
import numpy as np
import random
import time
import datetime

from tqdm import tqdm
tqdm.pandas()

from pathlib import Path
PATH_TO_DATA = Path("../data")
PATH_TO_MODELS = Path("../models")

/home/sakar/anaconda3/envs/MasterPiecer/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
tokenizer = BertTokenizer.from_pretrained(
    str(PATH_TO_MODELS / "rubert_cased_L-12_H-768_A-12_pt"),
    do_lower_case=False
)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.get_device_name(device)

'GeForce GTX 1060'

In [5]:
model = torch.load(str(PATH_TO_MODELS / "rater_trained.dump")).to(device)

In [6]:
def predict_score(synopsis):
    encoded_dict = tokenizer.encode_plus(
        synopsis,
        add_special_tokens=True,
        max_length=320,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    return model(encoded_dict['input_ids'].to(device),
                 attention_mask=encoded_dict['attention_mask'].to(device))[0].cpu().item()

In [7]:
synopses = pd.read_csv(PATH_TO_DATA / "synops.csv")
synopses

,synopsis
0,История жизни и карьеры основателя одной из са...
1,Наёмный убийца и его бывшая напарница решают п...
2,Фильм рассказывает историю о двух старых прият...
3,После смерти мужа молодая женщина Анна пытаетс...
4,В недалеком будущем человечество захватило Мар...
...,...
9995,История о двух молодых парнях из пригорода Май...
9996,Главный герой молодой человек из маленького го...
9997,В поисках приключений трое друзей отправляются...
9998,Наемник Джек Воробей оказывается втянутым в ра...


In [9]:
synopses['score'] = synopses['synopsis'].progress_apply(predict_score)

100%|██████████| 10000/10000 [06:15<00:00, 26.62it/s]


In [11]:
synopses.to_csv(PATH_TO_DATA / "result.csv", index=False)
